In [3]:
# importing required libraries
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.datasets import mnist

# loading dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# selecting a subset of data (200 images)
x_train = x_train[:200]
y = y_train[:200]

X = x_train.T
X = X/255

y.resize((200,1))
y = y.T

#checking value
pd.Series(y[0]).value_counts()

Using TensorFlow backend.


11493376/11490434 [==============================] - 2s 0us/step


1    26
9    23
7    21
4    21
3    21
0    21
2    20
6    19
8    15
5    13
dtype: int64

In [4]:
for i in range(y.shape[1]):
    if y[0][i] >4:
        y[0][i] = 1
    else:
        y[0][i] = 0

#checking value counts
pd.Series(y[0]).value_counts()

0    109
1     91
dtype: int64

In [5]:
# initializing filter
f=np.random.uniform(size=(3,5,5))
f = f.T

print('Filter 1', '\n', f[:,:,0], '\n')
print('Filter 2', '\n', f[:,:,1], '\n')
print('Filter 3', '\n', f[:,:,2], '\n')

Filter 1 
 [[0.57502042 0.22325089 0.71578171 0.03778955 0.2559068 ]
 [0.92193656 0.53760536 0.33264747 0.23387162 0.42855796]
 [0.65048127 0.32652839 0.92918415 0.31473543 0.18359411]
 [0.91596837 0.303635   0.19541383 0.78648483 0.34701774]
 [0.64211711 0.28309555 0.10967762 0.52172391 0.15691246]] 

Filter 2 
 [[0.49483126 0.77040679 0.99842582 0.54631572 0.29833313]
 [0.31012643 0.88187354 0.32967804 0.36269582 0.97048101]
 [0.8194949  0.72722713 0.23524152 0.83557025 0.2998311 ]
 [0.8723246  0.51499572 0.65407244 0.84430618 0.05077933]
 [0.78550106 0.61836234 0.06964925 0.2544751  0.27461226]] 

Filter 3 
 [[0.22493992 0.77910576 0.09628564 0.24745695 0.93514394]
 [0.6495476  0.75315443 0.56453427 0.93415683 0.07005371]
 [0.41624842 0.58567497 0.69904134 0.58880099 0.98218105]
 [0.76194524 0.6772001  0.97020859 0.38606932 0.66852158]
 [0.80521356 0.24960494 0.91680195 0.15167738 0.46636111]] 



In [6]:
X.shape, y.shape, f.shape

((28, 28, 200), (1, 200), (5, 5, 3))

In [7]:
# Convolution
new_image = []

# for number of images
for k in range(X.shape[2]):
    # sliding in horizontal direction
    for i in range(X.shape[0]-f.shape[0]+1):
        # sliding in vertical direction
        for j in range(X.shape[1]-f.shape[1]+1):
            new_image.append(X[:,:,k][i:i+f.shape[0],j:j+f.shape[1]])
            
# resizing the generated patches as per number of images
new_image = np.array(new_image)
new_image.resize((X.shape[2],int(new_image.shape[0]/X.shape[2]),new_image.shape[1],new_image.shape[2]))
new_image.shape

(200, 576, 5, 5)

In [8]:
# number of features in data set
s_row = X.shape[0] - f.shape[0] + 1
s_col = X.shape[1] - f.shape[1] + 1
num_filter = f.shape[2]

inputlayer_neurons = (s_row)*(s_col)*(num_filter)
output_neurons = 1 

# initializing weight
wo=np.random.uniform(size=(inputlayer_neurons,output_neurons))

In [9]:
# defining the Sigmoid Function
def sigmoid (x):
    return 1/(1 + np.exp(-x))

# derivative of Sigmoid Function
def derivatives_sigmoid(x):
    return x * (1 - x)

In [10]:
# generating output of convolution layer
filter_output = []
# for each image
for i in range(len(new_image)):
    # apply each filter
    for k in range(f.shape[2]):
        # do element wise multiplication
        for j in range(new_image.shape[1]):
            filter_output.append((new_image[i][j]*f[:,:,k]).sum()) 

filter_output = np.resize(np.array(filter_output), (len(new_image),f.shape[2],new_image.shape[1]))

# applying activation over convolution output
filter_output_sigmoid = sigmoid(filter_output)

filter_output.shape, filter_output_sigmoid.shape

((200, 3, 576), (200, 3, 576))

In [11]:
# generating input for fully connected layer
filter_output_sigmoid = filter_output_sigmoid.reshape((filter_output_sigmoid.shape[0],filter_output_sigmoid.shape[1]*filter_output_sigmoid.shape[2]))

filter_output_sigmoid = filter_output_sigmoid.T

# Linear trasnformation for fully Connected Layer
output_layer_input= np.dot(wo.T,filter_output_sigmoid)
output_layer_input = (output_layer_input - np.average(output_layer_input))/np.std(output_layer_input)

# activation function
output = sigmoid(output_layer_input)

In [12]:
#Error
error = np.square(y-output)/2

#Error w.r.t Output (Gradient)
error_wrt_output = -(y-output)

#Error w.r.t sigmoid transformation (output_layer_input)
output_wrt_output_layer_input=output*(1-output)

#Error w.r.t weight
output_wrt_w=filter_output_sigmoid

In [13]:
#delta change in w for fully connected layer
lr=15
delta_error_fcp = np.dot(output_wrt_w,(error_wrt_output * output_wrt_output_layer_input).T)

wo = wo - lr*delta_error_fcp

In [14]:
#Error w.r.t sigmoid output
output_layer_input_wrt_filter_output_sigmoid = wo.T

#Error w.r.t sigmoid transformation
filter_output_sigmoid_wrt_filter_output = filter_output_sigmoid * (1-filter_output_sigmoid)

# cvalculating derivatives for backprop convolution
error_wrt_filter_output = np.dot(output_layer_input_wrt_filter_output_sigmoid.T,error_wrt_output*output_wrt_output_layer_input) * filter_output_sigmoid_wrt_filter_output
error_wrt_filter_output = np.average(error_wrt_filter_output, axis=1)
error_wrt_filter_output = np.resize(error_wrt_filter_output,(X.shape[0]-f.shape[0]+1,X.shape[1]-f.shape[1]+1, f.shape[2]))

In [15]:
filter_update = []
for i in range(f.shape[2]):
    for j in range(f.shape[0]):
        for k in range(f.shape[1]):            
            temp = 0
            spos_row = j
            spos_col = k
            epos_row = spos_row + s_row
            epos_col = spos_col + s_col
            for l in range(X.shape[2]):
                temp = temp + (X[spos_row:epos_row,spos_col:epos_col,l]*error_wrt_filter_output[:,:,i]).sum()
            filter_update.append(temp/X.shape[2])  

filter_update_array = np.array(filter_update)
filter_update_array = np.resize(filter_update_array,(f.shape[2],f.shape[0],f.shape[1]))


In [16]:
for i in range(f.shape[2]):
    f[:,:,i] = f[:,:,i] - lr*filter_update_array[i]